In [14]:
import psycopg2 as p
import glob, os, csv, time
from datetime import datetime

## Updating Fleet totals

#### Analyst will need to put changes they make in the 'note' column in order for the ETL to know what to change

In [ ]:
# Updates the fleet and fleet_primary table from manual_review_input
# Fleet discrepancy will update all fleet totals, drivers, and vehicle miles
cur.execute("""
with updated as (
    update fleetseek.fleet
    set 
        trucks_owned = sub_q.trucks_owned,
        trucks_leased = sub_q.trucks_leased,
        tractors_owned = sub_q.tractors_owned,
        tractors_leased = sub_q.tractors_leased,
        trailers_owned = sub_q.trailers_owned,
        trailers_leased = sub_q.trailers_leased,
        haz_trailers_owned = sub_q.haz_trailers_owned,
        haz_trailers_leased = sub_q.haz_trailers_leased,
        haz_trucks_owned = sub_q.haz_trucks_owned,
        haz_trucks_leased = sub_q.haz_trucks_leased,
        total_trucks = sub_q.total_trucks,
        total_tractors = sub_q.total_tractors,
        total_trailers = sub_q.total_trailers,
        total_vehicles = sub_q.total_vehicles,
        total_vehicles_owned = sub_q.total_vehicles_owned,
        total_vehicles_leased = sub_q.total_vehicles_leased,
        dot_hmind = sub_q.hm_ind,
        year_mileage = sub_q.mcs150mileageyear,
        vehicle_miles = sub_q.mlg150,
        drivers = sub_q.tot_drs,
        cdl_drivers = sub_q.cdl_drivers,
        update_date = now()::date,
        mcn = sub_q.mcn,
        sic_code = sub_q.sic_code,
        product_code = sub_q.product_code,
        gvw_class_12 = sub_q.gvw_class_12,
        gvw_class_345 = sub_q.gvw_class_345,
        gvw_class_6 = sub_q.gvw_class_6,
        gvw_class_7 = sub_q.gvw_class_7,
        gvw_class_8 = sub_q.gvw_class_8,
        website = sub_q.website
        
        
    from (
        select 
            fs_trucks_owned as trucks_owned, 
            fs_trucks_leased as trucks_leased,
            fs_tractors_owned as tractors_owned,
            fs_tractors_leased as tractors_leased,
            fs_trailers_owned as trailers_owned,
            fs_trailers_leased as trailers_leased,
            fs_haz_trailers_owned as haz_trailers_owned,
            fs_haz_trailers_leased as haz_trailers_leased,
            fs_haz_trucks_owned as haz_trucks_owned,
            fs_haz_trucks_leased as haz_trucks_leased,
            sum(fs_trucks_owned + fs_trucks_leased + fs_haz_trucks_owned + fs_haz_trucks_leased) as total_trucks,
            sum(fs_tractors_owned + fs_tractors_leased) as total_tractors,
            sum(fs_trailers_owned + fs_trailers_leased + fs_haz_trailers_owned + fs_haz_trailers_leased) as total_trailers,
            sum(fs_trucks_owned + fs_trucks_leased + fs_tractors_owned + fs_tractors_leased +fs_haz_trucks_owned + fs_haz_trucks_leased) as total_vehicles,
            sum(fs_trucks_owned + fs_tractors_owned + fs_haz_trucks_owned) as total_vehicles_owned,
            sum(fs_trucks_leased + fs_tractors_leased + fs_haz_trucks_leased) as total_vehicles_leased,
            note,
            fs_fleet_id,
            hm_ind,
            mcs150mileageyear,
            mlg150,
            tot_drs,
            cdl_drivers,
            icc1 as mcn,
            fs_sic_code as sic_code,
            fs_product_code as product_code,
            fs_gvw_class_12 as gvw_class_12,
            fs_gvw_class_345 gvw_class_345,
            fs_gvw_class_6 gvw_class_6,
            fs_gvw_class_7 gvw_class_7,
            fs_gvw_class_8 gvw_class_8,
            fs_website as website

        from fleetseek.manual_review_input
        where 
            note like '%Fleet Discrepancy%'
        group by 
            fs_trucks_owned,
            fs_trucks_leased,
            fs_tractors_owned,
            fs_tractors_leased,
            fs_trailers_owned,
            fs_trailers_leased,
            fs_haz_trailers_owned,
            fs_haz_trailers_leased,
            fs_haz_trucks_owned,
            fs_haz_trucks_leased,
            update_reason,
            fs_fleet_id,
            mcmis_hm_ind,
            mcmis_mcs150mileageyear,
            mcmis_mlg150,
            mcmis_tot_drs,
            mcmis_cdl_drs,
            icc1,
            fs_sic_code,
            fs_product_code,
            fs_gvw_class_12,
            fs_gvw_class_345,
            fs_gvw_class_6,
            fs_gvw_class_7,
            fs_gvw_class_8,
            
    ) as sub_q
    where fleet.fleet_id = sub_q.fs_fleet_id
    returning
        fleet.total_vehicles,
        fleet.total_trailers,
        fleet.haz_trailers_owned,
        fleet.haz_trailers_leased,
        fleet.haz_trucks_owned,
        fleet.haz_trucks_leased,
        fleet.fleet_id
)
update fleetseek.fleet_primary
set
    total_vehicles = sub_a.total_vehicles,
    total_trailers = sub_a.total_trailers,
    hazmat_trucks = sub_a.hazmat_trucks,
    hazmat_trailers = sub_a.hazmat_trailers,
    update_date = now()::date
from (
    select 
        total_vehicles,
        total_trailers,
        sum(haz_trucks_owned + haz_trucks_leased) as hazmat_trucks,
        sum(haz_trailers_owned + haz_trailers_leased) as hazmat_trailers,
        fleet_id
    from updated
    group by 
        total_vehicles,
        total_trailers,
        fleet_id
    ) as sub_a
where fleet_primary.fleet_id = sub_a.fleet_id

""")
conn.commit()

## Changes to the company name

In [ ]:
# Fill in the FS_company_name field
cur.execute("""
update fleetseek.fleet 
set
    company_name = sub_q.company_name,
    company_sort_key = sub_q.company_sort_key
from (
    select 
        fs_company_name as company_name,
        upper(replace(replace(replace(replace(fs_company_name, ' ', ''), '.', ''), '-', ''), '&', '')) as company_sort_key,
        fs_fleet_id
    from fleetseek.manual_review_input
    where 
        note like '%Company Name Update%'
) as sub_q
where fleet.fleet_id = sub_q.fs_fleet_id


""")

## Changes to phone numbers

In [ ]:
# Updates regular phone numbers, fax numbers, and toll free numbers
cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_phone_number as phone_number,
        fs_phone_id
    from fleetseek.manual_review_input
    where
        note like '%Update Phone%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_phone_id;


update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_fax_number,
        fs_fax_id
    from fleetseek.manual_review_input
    where
        note like '%Fax%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_fax_id;


update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_tf_number,
        fs_toll_free_id
    from fleetseek.manual_review_input
    where
        note like '%TF%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_toll_free_id;

""")
conn.commit()

## Location changes

In [ ]:
# Specifically for physical address changes
# Going forward updates will exclusively use the address2 field
cur.execute("""

update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = null,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    updated_date_time = now(),
    latitude = sub_q.latitude,
    longitude = sub_q.longitude,
    county_name = sub_q.county_name
from (
    select
        fs_phy_address1 as address1,
        fs_phy_address2 as address2,
        fs_phy_city as city,
        fs_phy_state_code as state_code,
        fs_phy_zip_code as zip_code,
        fs_phy_latitude as latitude,
        fs_phy_longitude as longitude,
        fs_phy_county_name as county_name,
        fs_physical_location_id as location_id,
        fs_phy_county_fips as county_fips
    from fleetseek.manual_review_input 
    where note like '%Physical Address Change%'
) as sub_q
where location.location_id = sub_q.location_id
""")
conn.commit()


In [ ]:
# Mailing address changes
# Going forward updates will exclusively use the address2 field
cur.execute("""

update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = null,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    updated_date_time = now(),
    latitude = sub_q.latitude,
    longitude = sub_q.longitude,
    county_name = sub_q.county_name
from (
    select
        fs_mai_address1 as address1,
        fs_mai_address2 as address2,
        fs_mai_city as city,
        fs_mai_state_code as state_code,
        fs_mai_zip_code as zip_code,
        fs_mai_latitude as latitude,
        fs_mai_longitude as longitude,
        fs_mai_county_name as county_name,
        fs_mailing_location_id as location_id,
        fs_mai_county_fips as county_fips
    from fleetseek.manual_review_input 
    where update_reason like '%Mailing Address Change%'
) as sub_q
where location.location_id = sub_q.location_id
""")
conn.commit()


In [ ]:
cur.execute("""
alter table fleetseek.contact
add column fleet_id integer
""")
conn.commit()

## New contacts

In [ ]:
# New contacts that have the same location ids as other contacts in the fleet
# Analyst will have to create new rows for contacts that have the same location information

cur.execute("""
with inserted as (
    insert into fleetseek.contact (honorific, first_name, last_name, middle_name, suffix, physical_location_id, mailing_location_id, original_job_title, job_title_id, fleet_id)
    select
        fs_honorific2,
        fs_first_name2,
        fs_last_name2,
        fs_middle_name2,
        fs_suffix_2,
        fs_physical_location_id,
        fs_mailing_location_id,
        fs_original_job_title2,
        fs_job_title_id2,
        fs_fleet_id
    from fleetseek.manual_review_input
    where 
        note like '%Contact%' and
        fs_contact2_id is null and 
        fs_first_name2 is not null 
    returning
        contact_id,
        fleet_id
)
insert into fleetseek.fleet_contact (fleet_id, contact_id, sort_order, active)
select
    fleet_id,
    contact_id, 
    2,
    true
from inserted;

alter table fleetseek.contact
drop column fleet_id
""")
conn.commit()

## Commodities

In [ ]:
# Delete from first before inserting new commodities
cur.execute("""
DELETE 
FROM fleetseek.fleet_commodity
WHERE fleet_id in (select fs_fleet_id
                  from fleetseek.manual_review_input
                  where note like '%Commodity%')
""")
conn.commit()

In [ ]:
# Will insert commodities from manual review and then update the commodity_id field in the fleet_primary table
cur.execute("""
with inserted as (
    insert into fleetseek.fleet_commodity (fleet_id, commodity_id, sort_order)
    select sub_q.fleet_id, sub_q.c_commodity_id, sub_q.sort_order
    from (

        select mri.fs_fleet_id as fleet_id, mri.fs_commodities as fp_commodity_id, c.commodity_id as c_commodity_id, c.sort_order
        from fleetseek.manual_review_input as mri
        join fleetseek.commodity as c 
        on c.commodity_id = any(string_to_array(mri.fs_commodities, ', ')::integer[])
        where mri.note like '%Commodity%'
    ) as sub_q
    returning
        fleet_id,
        commodity_id
)
update fleetseek.fleet_primary
set
    commodity_id = sub_q.commodity_id
from (
    select string_to_array(aggregate, ', ')::integer[] as commodity_id, fleet_id
    from (

        select string_agg(commodity_id::varchar, ', ') as aggregate, fleet_id
        from inserted
        group by fleet_id
    ) as sub_a
) as sub_q
where fleet_primary.fleet_id = sub_q.fleet_id
""")

## Trailer types

In [ ]:
# Delete from fleet_trailer_type first before inserting new trailer_info
cur.execute("""
DELETE 
FROM fleetseek.fleet_trailer_type
WHERE fleet_id in (select fs_fleet_id
                  from fleetseek.manual_review_input
                  where note like '%Trailer Type%')
""")
conn.commit()

In [ ]:
cur.execute("""
with inserted as (
    insert into fleetseek.fleet_trailer_type (fleet_id, trailer_type_id, sort_order)
    select sub_q.fleet_id, sub_q.tt_trailer_type_id, sub_q.sort_order
    from (

        select mri.fs_fleet_id as fleet_id, mri.fs_trailer_types as trailer_type_id, tt.trailer_type_id as tt_trailer_type_id, tt.sort_order
        from fleetseek.manual_review_input as mri
        join fleetseek.trailer_type as tt
        on tt.trailer_type_id = any(string_to_array(mri.fs_trailer_types, ', ')::integer[])
        where mri.note like '%Trailer Type%'
    ) as sub_q
    returning
        fleet_id,
        trailer_type_id
)
update fleetseek.fleet_primary
set
    trailer_type_id = sub_q.trailer_type_id
from (
    select string_to_array(aggregate, ', ')::integer[] as trailer_type_id, fleet_id
    from (

        select string_agg(trailer_type_id::varchar, ', ') as aggregate, fleet_id
        from inserted
        group by fleet_id
    ) as sub_a
) as sub_q
where fleet_primary.fleet_id = sub_q.fleet_id

""")

## Contact name info and job info

In [ ]:
# Need IDs for contact, email, and all phone numbers
cur.execute("""

with updated as (
    update fleetseek.contact
    set 
        honorific = sub_q.honorific,
        first_name = sub_q.first_name,
        last_name = sub_q.last_name,
        middle_name = sub_q.middle_name,
        suffix = sub_q.suffix,
        updated_date_time = now()::date,
        original_job_title = sub_q.original_job_title,
        active = sub_q.active_contact,
        job_title_id = sub_q.job_title_id
    from (
        select 
            honorific,
            first_name,
            last_name,
            middle_name,
            suffix,
            original_job_title,
            active_contact,
            job_title_id,
            contact_id as manual_review_contact_id
        from fleetseek.manual_review_contact_input
        where note like '%Update Contact%'
    ) as sub_q
    where contact.contact_id = sub_q.manual_review_contact_id
    returning
        active,
        contact_id
)
update fleetseek.fleet_contact
set
    active = updated.active
from updated
where fleet_contact.contact_id = updated.contact_id
"""
)
conn.commit()

## Contact email

In [ ]:
cur.execute("""
update fleetseek.email
set
    email_address = sub_q.email_address
from (
    select 
        email_address,
        email_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Email%'
) as sub_q
where email.email_id = sub_q.email_id
"""
)
conn.commit()

## Contact phone info

In [ ]:
cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Phone%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Fax%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.manual_review_contact_input
    where note like '%Update TF%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

""")
conn.commit()

## Contact address

In [ ]:
cur.execute("""
update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = sub_q.address3,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    country_code = sub_q.country_code,
    updated_date_time = now(),
    active = sub_q.active,
    county_name = sub_q.county_name
from (
    select 
        phy_address as address1,
        phy_address2 as address2,
        phy_address3 as address3,
        phy_city as city,
        phy_countyfips as county_fips,
        phy_state_code as state_code,
        phy_zip_code as zip_code,
        phy_country_code as country_code,
        phy_active as active,
        phy_county_name as county_name,
        phy_location_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Physical Address%'

) as sub_q
where location.location_id = sub_q.phy_location_id;


update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = sub_q.address3,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    country_code = sub_q.country_code,
    updated_date_time = now(),
    active = sub_q.active,
    county_name = sub_q.county_name
from (
    select 
        mai_address as address1,
        mai_address2 as address2,
        mai_address3 as address3,
        mai_city as city,
        mai_countyfips as county_fips,
        mai_state_code as state_code,
        mai_zip_code as zip_code,
        mai_country_code as country_code,
        mai_active as active,
        mai_county_name as county_name,
        mai_location_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Mailing Address%'

) as sub_q
where location.location_id = sub_q.mai_location_id;
""")
conn.commit()